# Quadrotor Example
This notebook will demonstrate how to set up and solve a trajectory optimization problem for a quadrotor. In particular, it will highlight how TrajectoryOptimization.jl accounts for the group structure of 3D rotations.

### Loading the Required Packages
To define the quadrotor model, we import `RobotDynamics` and `Rotations`, and use `TrajectoryOptimization` to define the problem. We load in `StaticArrays` and `LinearAlgebra` to help with the setup.

In [1]:
# import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

In [1]:
using RobotDynamics, Rotations
using TrajectoryOptimization
using StaticArrays, LinearAlgebra

## Creating the model
We could use the quadrotor model defined in `RobotZoo.jl`, but instead we'll go through the details of using the `RigidBody` interface in `RobotDyanmics`.

We start by defining our new `Quadrotor` type, which inherits from `RigidBody{R}`:

In [2]:
struct Quadrotor{R} <: RigidBody{R}
    n::Int
    m::Int
    mass::Float64
    J::Diagonal{Float64,SVector{3,Float64}}
    Jinv::Diagonal{Float64,SVector{3,Float64}}
    gravity::SVector{3,Float64}
    motor_dist::Float64
    kf::Float64
    km::Float64
    bodyframe::Bool  # velocity in body frame?
    info::Dict{Symbol,Any}
end

function Quadrotor{R}(;
        mass=0.5,
        J=Diagonal(@SVector [0.0023, 0.0023, 0.004]),
        gravity=SVector(0,0,-9.81),
        motor_dist=0.1750,
        kf=1.0,
        km=0.0245,
        bodyframe=false,
        info=Dict{Symbol,Any}()) where R
    Quadrotor{R}(13,4,mass,J,inv(J),gravity,motor_dist,kf,km,bodyframe,info)
end

(::Type{Quadrotor})(;kwargs...) = Quadrotor{UnitQuaternion{Float64}}(;kwargs...)

where `R` is the rotation parameterization being used, typically one of `UnitQuaternion{T}`, `MRP{T}`, or `RodriguesParam{T}`. 

We now need to define the number of control inputs:

In [3]:
RobotDynamics.control_dim(::Quadrotor) = 4

Now we are ready to define the dynamics of our quadrotor, which we do by simply defining the forces and moments acting on our quadrotor for a given state and control, as well as some "getter" methods for our inertial properties.

It's important to note that the force is in the world frame, and torque is in the body frame.

In [4]:
function RobotDynamics.forces(model::Quadrotor, x, u)
    q = orientation(model, x)
    kf = model.kf
    g = model.gravity
    m = model.mass

    # Extract motor speeds
    w1 = u[1]
    w2 = u[2]
    w3 = u[3]
    w4 = u[4]

    # Calculate motor forces
    F1 = max(0,kf*w1);
    F2 = max(0,kf*w2);
    F3 = max(0,kf*w3);
    F4 = max(0,kf*w4);
    F = @SVector [0., 0., F1+F2+F3+F4] #total rotor force in body frame

    m*g + q*F # forces in world frame
end

function RobotDynamics.moments(model::Quadrotor, x, u)

    kf, km = model.kf, model.km
    L = model.motor_dist

    # Extract motor speeds
    w1 = u[1]
    w2 = u[2]
    w3 = u[3]
    w4 = u[4]
    
    # Calculate motor forces
    F1 = max(0,kf*w1);
    F2 = max(0,kf*w2);
    F3 = max(0,kf*w3);
    F4 = max(0,kf*w4);

    # Calculate motor torques
    M1 = km*w1;
    M2 = km*w2;
    M3 = km*w3;
    M4 = km*w4;
    tau = @SVector [L*(F2-F4), L*(F3-F1), (M1-M2+M3-M4)] #total rotor torque in body frame
end

RobotDynamics.inertia(model::Quadrotor) = model.J
RobotDynamics.inertia_inv(model::Quadrotor) = model.Jinv
RobotDynamics.mass(model::Quadrotor) = model.mass

And with that our model is defined!

## Setting up our problem
For our trajectory optimization problem, we're going to have the quadrotor do a "zig-zag" pattern. We can do this via objective/cost function manipulation. We start by creating our quadrotor model and defining our integration scheme:

In [5]:
# Set up model and discretization
model = Quadrotor();
n,m = size(model)
N = 101                # number of knot points
tf = 5.0               # total time (sec)
dt = tf/(N-1)          # time step (sec)

0.05

In [6]:
n,m = size(model)

(13, 4)

We now need to set up the initial and final conditions for our quadrotor, which we want to move 20 meters in the x-direction. We can build the state piece-by-piece using the `RobotDynamics.build_state` function.

In [10]:
# x0_pos = SA[0, -10, 1.]
# xf_pos = SA[0, +10, 1.]
x0_pos = SA[-7, -10, 1.]
xf_pos = SA[5, +10, 4.]
x0 = RobotDynamics.build_state(model, x0_pos, UnitQuaternion(I), zeros(3), zeros(3))
xf = RobotDynamics.build_state(model, xf_pos, UnitQuaternion(I), zeros(3), zeros(3));

In [11]:
xf

13-element SArray{Tuple{13},Float64,1,13} with indices SOneTo(13):
  5.0
 10.0
  4.0
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

### Creating the cost function
We now create a cost function that encourages a "zig-zag" pattern for the quadrotor. We set up a few waypoints at specific times, and impose a high cost for being far from those locations.

In [14]:
# Set up waypoints
# wpts = [SA[+10, 0, 1.],
#         SA[-10, 0, 1.],
#         xf_pos]
# times = [33, 66, 101]   # in knot points
wpts = [xf_pos]
times = [100]

# Set up nominal costs q^T x + r^T u
# which are just zero because x_nom
Q = Diagonal(RobotDynamics.fill_state(model, 1e-5, 1e-5, 1e-3, 1e-3))
R = Diagonal(@SVector fill(1e-4, 4))
q_nom = UnitQuaternion(I)
v_nom = zeros(3)
ω_nom = zeros(3)
x_nom = RobotDynamics.build_state(model, zeros(3), q_nom, v_nom, ω_nom)
cost_nom = LQRCost(Q, R, x_nom)  # u=zeros() default

DiagonalCost{13,4,Float64}([1.0e-5 0.0 … 0.0 0.0; 0.0 1.0e-5 … 0.0 0.0; … ; 0.0 0.0 … 0.001 0.0; 0.0 0.0 … 0.0 0.001], [0.0001 0.0 0.0 0.0; 0.0 0.0001 0.0 0.0; 0.0 0.0 0.0001 0.0; 0.0 0.0 0.0 0.0001], [-0.0, -0.0, -0.0, -1.0e-5, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0], 5.0e-6, false)

In [15]:
# Set up waypoint costs
Qw_diag = RobotDynamics.fill_state(model, 1e3,1,1,1)
Qf_diag = RobotDynamics.fill_state(model, 10., 100, 10, 10)
costs = map(1:length(wpts)) do i
    r = wpts[i]
    xg = RobotDynamics.build_state(model, r, q_nom, v_nom, ω_nom)
    if times[i] == N
        Q = Diagonal(Qf_diag)
    else
        Q = Diagonal(1e-3*Qw_diag)
    end

    LQRCost(Q, R, xg)
end

1-element Array{DiagonalCost{13,4,Float64},1}:
 DiagonalCost{13,4,Float64}([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.001 0.0; 0.0 0.0 … 0.0 0.001], [0.0001 0.0 0.0 0.0; 0.0 0.0001 0.0 0.0; 0.0 0.0 0.0001 0.0; 0.0 0.0 0.0 0.0001], [-5.0, -10.0, -4.0, -0.001, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0], 70.5005, false)

In [16]:
# Build Objective
costs_all = map(1:N) do k
    i = findfirst(x->(x ≥ k), times)
    if k ∈ times
        costs[i]
    else
        cost_nom
    end
end
obj = Objective(costs_all);

In [17]:
# Zigzag movement
print(x0[1:3])
wpts

[-7.0, -10.0, 1.0]

1-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [5.0, 10.0, 4.0]

In [18]:
# [x,x,x, q,q,q,q, v,v,v, ω,ω,ω]
RobotDynamics.fill_state(model, 1e-5, 1e-5, 1e-3, 1e-3)

13-element SArray{Tuple{13},Float64,1,13} with indices SOneTo(13):
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001

In [13]:
RobotDynamics.fill_state

fill_state (generic function with 2 methods)

### Initialization
We initialize the solver with a simple hover trajectory that keeps the quadrotor hovering at the initial position.

In [19]:
u0 = @SVector fill(0.5*model.mass/m, m)
U_hover = [copy(u0) for k = 1:N-1]; # initial hovering control trajectory

In [15]:
u0

4-element SArray{Tuple{4},Float64,1,4} with indices SOneTo(4):
 0.0625
 0.0625
 0.0625
 0.0625

### Constraints
For this problem, we only impose bounds on the controls.

In [20]:
conSet = ConstraintList(n,m,N)
bnd = BoundConstraint(n,m, u_min=0.0, u_max=12.0)
add_constraint!(conSet, bnd, 1:N-1)

In [17]:
print(conSet)

ConstraintList(13, 4, TrajectoryOptimization.AbstractConstraint[BoundConstraint{8,17,Float64}(13, 4, [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, 12.0, 12.0, 12.0, 12.0], [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, 0.0, 0.0, 0.0, 0.0], [105, 114, 123, 132], [109, 118, 127, 136], [14, 15, 16, 17, 31, 32, 33, 34])], UnitRange{Int64}[1:100], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0])

### Building the Problem
We now build the trajectory optimization problem, providing a dynamically-feasible initialization.

In [23]:
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet)
initial_controls!(prob, U_hover)
rollout!(prob);

## Solving the Problem using ALTRO
With our problem set up, can we solve it using any of the supported solvers. We'll use ALTRO:

In [25]:
using Altro
opts = SolverOptions(
    penalty_scaling=100.,
    penalty_initial=0.1,
)

solver = ALTROSolver(prob, opts);
solve!(solver)
# println("Cost: ", cost(solver))
# println("Constraint violation: ", max_violation(solver))
# println("Iterations: ", iterations(solver))

LoadError: [91mMethodError: no method matching cost_expansion!(::Objective{QuadraticCost{13,4,Float64,SizedArray{Tuple{13,13},Float64,2,2,Array{Float64,2}},SizedArray{Tuple{4,4},Float64,2,2,Array{Float64,2}}}}, ::Objective{DiagonalCost{13,4,Float64}}, ::Traj{13,4,Float64,RobotDynamics.GeneralKnotPoint{Float64,13,4,SArray{Tuple{17},Float64,1,17}}}, ::Bool, ::Bool)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  cost_expansion!(::Any, ::Objective, ::Traj, ::Any; init, rezero) at /home/alvin/.julia/packages/TrajectoryOptimization/ZrvuZ/src/cost.jl:121[39m
[91m[0m  cost_expansion!(::Objective{QuadraticCost{n,m,T,SizedArray{Tuple{n,n},T,2,2,Array{T,2}},SizedArray{Tuple{m,m},T,2,2,Array{T,2}}}} where T where m where n, [91m::Altro.ALObjective[39m, ::Traj, ::Bool, ::Bool) at /home/alvin/.julia/packages/Altro/ZbP1c/src/augmented_lagrangian/al_objective.jl:35[39m
[91m[0m  cost_expansion!(::Any, ::Objective, ::Traj) at /home/alvin/.julia/packages/TrajectoryOptimization/ZrvuZ/src/c

## Visualizing the solution
We can use `TrajOptPlots` to visualize the solution:

In [39]:
using TrajOptPlots
using MeshCat
using Plots

vis = Visualizer()
render(vis)

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/alvin/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

For the visualization, we use `MeshIO v0.3` and `FileIO` to load in a mesh file. For the visualization, we need to tell `TrajOptPlots` what geometry to display, which we do by defining the `_set_mesh!` method for our model. Since our model is a `RigidBody`, `TrajOptPlots` already knows how to display it once the robot geometry is defined.

In [44]:
using FileIO, MeshIO, TrajOptPlots
function TrajOptPlots._set_mesh!(vis, model::Quadrotor)
    obj = joinpath(@__DIR__, "quadrotor.obj")
    quad_scaling = 0.085
    robot_obj = FileIO.load(obj)
    robot_obj *= quad_scaling
    mat = MeshPhongMaterial(color=colorant"black")
    setobject!(vis["geom"], robot_obj, mat)
end
TrajOptPlots.set_mesh!(vis, model)

MeshCat Visualizer with path /meshcat/robot/geom at http://127.0.0.1:8700

In [46]:
visualize!(vis, solver);